In [51]:
import helpers as h

import numpy as np
import pandas as pd
from scipy.optimize import minimize, NonlinearConstraint, BFGS

from icecream import ic

In [2]:
# example 
def rosen(x):
    """The Rosenbrock function"""
    return sum(100.0*(x[1:]-x[:-1]**2.0)**2.0 + (1-x[:-1])**2.0)

x0 = np.array([1.3, 0.7, 0.8, 1.9, 1.2])

res = minimize(rosen, x0, method='nelder-mead',
               options={'xatol': 1e-8, 'disp': True})

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 339
         Function evaluations: 571


In [3]:
# constants 
# states_path = "data/states/nga_admbnda_adm1_osgof_20161215.shp"
# states = gpd.read_file(states_path)

data_path_names = ["wind_speed", "temperature", "solar_flux"]

data_averages = {}
for name in data_path_names:
    data_averages[name] = pd.read_csv(f"clean_data/{name}.csv", index_col=0)

avg_vals = {n: i["value"] for n, i in data_averages.items()}

In [4]:
state_areas = pd.read_csv(f"clean_data/state_areas.csv")

In [5]:
def get_constants():
    data_path_names = ["wind_speed", "temperature", "solar_flux"]

    data_averages = {}
    for name in data_path_names:
        data_averages[name] = pd.read_csv(f"clean_data/{name}.csv", index_col=0)

    avg_vals = {n: i["value"] for n, i in data_averages.items()}

    state_areas = pd.read_csv(f"clean_data/state_areas.csv")

    F_cur=avg_vals["solar_flux"]
    T_a=avg_vals["temperature"]
    w=avg_vals["wind_speed"]

    return (F_cur, T_a, w, state_areas)

    

In [6]:
# F_cur, T_a, w, state_areas = get_constants()


In [7]:
def do_calc(x, constants):
    """ 
    - pl_solar, pl_wind ~ percent of land 
    - ns_solar, ns_wind ~ number of states 
    """

    pl_solar, pl_wind, ns_solar, ns_wind = x
    # ic(pl_solar, pl_wind, ns_solar, ns_wind )
    

    # constant values 
    F_cur, T_a, w, state_areas = constants[0]
    
    # solar capacity for each state 
    P_ac = h.calculate_power_panel(F_cur, T_a, w)

    # return P_ac

    # wind capacity for each state 
    P_m, P_t, CF, P_t_after_losses, P_r = h.calculate_power_turbine(V_m=w)

    # panel distribution 
    solar_res = h.calc_num_panels(pl_solar, 
        ns_wind, 
        state_areas, 
        P_ac)

    # turbine distribution 
    wind_res = h.calc_num_turbines(pl_wind, 
        ns_wind, 
        state_areas, 
        P_t_after_losses, 
        P_r)

    return solar_res, wind_res

In [8]:
int(4.0)

4

In [42]:
class optimize_energy():
    def __init__(self):
        c = get_constants()
        self.F_cur = c[0]
        self.T_a = c[1]
        self.w = c[2] 
        self.state_areas = c[3]

    def do_calc(self, x):
        """ 
        - pl_solar, pl_wind ~ percent of land 
        - ns_solar, ns_wind ~ number of states 
        """
        pl_solar, pl_wind, ns_solar, ns_wind = x

        # capacities for each state 
        P_ac = h.calculate_power_panel(self.F_cur, self.T_a, self.w)
        P_m, P_t, CF, P_t_after_losses, P_r = h.calculate_power_turbine(V_m=self.w)

        # distributions
        solar_res = h.calc_num_panels(pl_solar, 
            ns_wind, 
            self.state_areas, 
            P_ac)
        wind_res = h.calc_num_turbines(pl_wind, 
            ns_wind, 
            self.state_areas, 
            P_t_after_losses, 
            P_r)
        self.solar_res = solar_res
        self.wind_res = wind_res 

        return solar_res, wind_res

    def min_devices(self, x):
        solar_res, wind_res = self.do_calc(x)
        self.num_devices = solar_res["n_panels"].sum() + wind_res["n_turbines"].sum()

        return self.num_devices

    def meet_energy(self, x):
        solar_res, wind_res = self.do_calc(x)
        self.total_energy = solar_res["total_energy (mwh)"] + wind_res["total_energy (mwh)"]
        self.energy_need = 26_5337_353 # 27 million MWh 
        self.dif = self.total_energy - self.energy_need # > 0 

        return self.dif 
    

In [58]:
o = optimize_energy()
x0 = [0.02, 0.02, 4, 4]

nlc  = NonlinearConstraint(fun=o.meet_energy, lb=0, ub=1e3, jac='2-point', hess=BFGS())
res = minimize(fun=o.min_devices, x0=x0, constraints=nlc, method='trust-constr', options={'disp': True, 'maxiter': 300})

/Users/julietnwagwuume-ezeoke/_ClassCode/classenv/lib/python3.9/site-packages/scipy/optimize/_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


The maximum number of function evaluations is exceeded.
Number of iterations: 300, function evaluations: 1670, CG iterations: 290, optimality: 1.53e-05, constraint violation: 4.39e+00, execution time:  9.9 s.


In [59]:
o.dif

-4.385694205760956

In [61]:
res.x

array([1.63255237e-04, 2.00000000e-02, 4.00000000e+00, 4.00000000e+00])

In [62]:
o = optimize_energy()
x0 = [0.1, 0.1, 30, 30]

nlc  = NonlinearConstraint(fun=o.meet_energy, lb=0, ub=1e3, jac='2-point', hess=BFGS())
res = minimize(fun=o.min_devices, x0=x0, constraints=nlc, method='trust-constr', options={'disp': True, 'maxiter': 300})

/Users/julietnwagwuume-ezeoke/_ClassCode/classenv/lib/python3.9/site-packages/scipy/optimize/_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


`xtol` termination condition is satisfied.
Number of iterations: 284, function evaluations: 2610, CG iterations: 316, optimality: 1.22e-04, constraint violation: 3.32e+00, execution time: 1.6e+01 s.


In [63]:
res.x

array([-6.92195248e-04,  1.00000000e-01,  3.00000000e+01,  3.00000000e+01])

In [65]:
o.dif

-3.319736957550049

In [66]:
o = optimize_energy()
x0 = [0.3, 0.4, 10, 30]

nlc  = NonlinearConstraint(fun=o.meet_energy, lb=0, ub=0, jac='2-point', hess=BFGS())
res = minimize(fun=o.min_devices, x0=x0, constraints=nlc, method='trust-constr', options={'disp': True, 'maxiter': 300})

/Users/julietnwagwuume-ezeoke/_ClassCode/classenv/lib/python3.9/site-packages/scipy/optimize/_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


`xtol` termination condition is satisfied.
Number of iterations: 19, function evaluations: 70, CG iterations: 18, optimality: 0.00e+00, constraint violation: 7.03e+00, execution time: 0.45 s.


In [67]:
res.x

array([-1.00493545e-02,  1.33735405e+00,  1.00000000e+01,  3.00000000e+01])

- things to minimize potentially
  - total land used 
  - number of turbines + number of panels  ~ cost 
- start by definining seperate function 

- [mnimize where output has contraints](https://stackoverflow.com/questions/64169852/optimizing-input-where-output-has-constraints-in-python)
- 
- [use a class for constants](https://stackoverflow.com/questions/64205381/can-i-send-arguments-to-a-constraint-function-using-scipy-optimize-nonlinearcons)

In [68]:
o.dif

-10.796165466308594

In [69]:
o.num_devices

-5623709918.0

In [ ]:
# things to minimize potentially 
    